In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
arc_train = zipfile.ZipFile('/kaggle/input/whats-cooking/train.json.zip','r')
train_df = pd.read_json(arc_train.read('train.json'))
train_df.head()

In [ ]:
arc_test = zipfile.ZipFile('/kaggle/input/whats-cooking/test.json.zip','r')
test_df = pd.read_json(arc_test.read('test.json'))
test_df.head()

In [ ]:
train_df.info()

In [ ]:
test_df.info()

In [ ]:
train_df['cuisine'].value_counts()

In [ ]:
#train_df['clean_ingredients']=[' , '.join(z).strip() for z in train_df['ingredients']]
#test_df['clean_ingredients']=[' , '.join(z).strip() for z in test_df['ingredients']]

In [ ]:
train_ingred_count = {}
for i in range(len(train_df)):
    for j in train_df['ingredients'][i]:
        if j in train_ingred_count.keys():
            train_ingred_count[j]+=1
        else:
            train_ingred_count[j]=1

In [ ]:
test_ingred_count = {}
for i in range(len(test_df)):
    for j in test_df['ingredients'][i]:
        if j in test_ingred_count.keys():
            test_ingred_count[j]+=1
        else:
            test_ingred_count[j]=1

In [ ]:
print(len(train_ingred_count))
print(len(test_ingred_count))

In [ ]:
ingred_miss_train=[]
for i in test_ingred_count.keys():
    if i not in train_ingred_count.keys():
        ingred_miss_train.append(i)
print(len(ingred_miss_train))

In [ ]:
for i in ingred_miss_train:
    train_ingred_count[i]=0
print(len(train_ingred_count))

In [ ]:
ingred_miss=[]
for i in train_ingred_count.keys():
    if i not in test_ingred_count.keys():
        ingred_miss.append(i)
print(len(ingred_miss))

In [ ]:
for i in ingred_miss:
    test_ingred_count[i]=0
print(len(test_ingred_count))

In [ ]:
for i in train_ingred_count.keys():
    train_df[i]=np.zeros(len(train_df))

In [ ]:
for i in test_ingred_count.keys():
    test_df[i]=np.zeros(len(test_df))

In [ ]:
print(train_df.shape)
print(test_df.shape)

In [ ]:
for i in range(len(train_df)):
    for j in train_df['ingredients'][i]:
        train_df[j].iloc[i]=1

In [ ]:
train_df.head()

In [ ]:
for i in range(len(test_df)):
    for j in test_df['ingredients'][i]:
        test_df[j].iloc[i]=1

In [ ]:
test_df.head()

In [ ]:
test_df=test_df[train_df.drop('cuisine', axis=1).columns]

In [ ]:
from sklearn.model_selection import train_test_split
X=train_df.drop(['id','ingredients','cuisine'],axis=1)
y=train_df['cuisine']

In [ ]:
X_train, X_val, y_train, y_val=train_test_split(X,y,random_state=42)

In [ ]:
print(X_train.shape,y_train.shape)
print(X_val.shape,y_val.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

In [ ]:
lr=LogisticRegression()
lr.fit(X_train,y_train)

In [ ]:
lr.score(X_val,y_val)

In [ ]:
test_df['cuisine']=lr.predict(test_df.drop(['id','ingredients'],axis=1))

In [ ]:
Submission=test_df[['id','cuisine']]
Submission.set_index('id',inplace=True)

In [ ]:
Submission.to_csv('Submission.csv')